In [ ]:
import akshare as ak
dd = ak.stock_zh_a_hist('300913')

In [ ]:
dd

In [ ]:
import akshare as ak
quarters = ["0331", "0630", "0930", "1231"]
date_list = [str(i) + q for i in range(2009, 2022) for q in quarters]
print(date_list)


stock_yjbb_em_df = ak.stock_yjbb_em(date="20220331")
print(stock_yjbb_em_df  )


In [ ]:

import talib as tl
import akshare as ak
from requests.exceptions import RequestException
import concurrent.futures
from pandas import DataFrame, concat

def fetch_one_quarter_bb_from_ak(date):
    """
    返回沪深京所有股票某季度的业绩报表数据
    Args: date
        type:str choice of {"XXXX0331", "XXXX0630", "XXXX0930", "XXXX1231"}; 从 20100331 开始
    Return: stock_data
        type:daraframe https://www.akshare.xyz/data/stock/stock.html#id136
    """

    try:
        data = ak.stock_yjbb_em(date=date)
    except RequestException as exc:
        print(f'request failed: unable to get stock data: {exc}')
        return False

    if data is None or data.empty:
        print(f'日期:{date},没有数据')
        return False

    return data


def fetch_all_quarter_bb_from_ak():
    """
    返回沪深京所有股票季度业绩报表数据
    Return: all_bb
        type:dataframe  https://www.akshare.xyz/data/stock/stock.html#id136
    """
    all_bb = DataFrame()
    quarters = ["0331", "0630", "0930", "1231"]
    date_list = [str(i) + q for i in range(2010, 2023) for q in quarters]

    with concurrent.futures.ThreadPoolExecutor(max_workers=12) as executor:
        future_to_one_quarters = {
            executor.submit(fetch_one_quarter_bb_from_ak, date):
            date
            for date in date_list
        }  
        
        # as_completed:当子线程中的任务执行完后，用 result() 获取返回结果
        for future in concurrent.futures.as_completed(future_to_one_quarters):
            try:
                data = future.result()
                # if data:
                all_bb = concat([all_bb, data], axis=0)

            except Exception as exc:
                print(f'future.result() generated an exception: {exc}')
    return all_bb


In [ ]:
all_bb = fetch_all_quarter_bb_from_ak()

In [ ]:
tt = all_bb[all_bb['股票代码'] == '600519']
# tt按照报告日期排序
# print(tt.head)
tt = tt.sort_values(by='最新公告日期', ascending=True)
tt

In [ ]:
import pandas as pd

df_merged = pd.DataFrame()
# 创建两个DataFrame
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                    'B': ['B0', 'B1', 'B2', 'B3'],
                    'C': ['C0', 'C1', 'C2', 'C3'],
                    'D': ['D0', 'D1', 'D2', 'D3']})

df2 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'],
                    'B': ['B4', 'B5', 'B6', 'B7'],
                    'C': ['C4', 'C5', 'C6', 'C7'],
                    'D': ['D4', 'D5', 'D6', 'D7']})

df3 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'],
                    'B': ['B4', 'B5', 'B6', 'B7'],
                    'C': ['C4', 'C5', 'C6', 'C7'],
                    'D': ['D4', 'D5', 'D6', 'D7']})

# 按行合并DataFrame
df_merged = pd.concat([df_merged, df1], axis=0)
df_merged = pd.concat([df_merged, df2], axis=0)

df_merged = pd.concat([df_merged, False], axis=0)


print(df_merged)

In [47]:
quarters = ["0331", "0630", "0930", "1231"]
date_list = [str(i) + q for i in range(2010, 2022) for q in quarters]
date_list   


['20100331',
 '20100630',
 '20100930',
 '20101231',
 '20110331',
 '20110630',
 '20110930',
 '20111231',
 '20120331',
 '20120630',
 '20120930',
 '20121231',
 '20130331',
 '20130630',
 '20130930',
 '20131231',
 '20140331',
 '20140630',
 '20140930',
 '20141231',
 '20150331',
 '20150630',
 '20150930',
 '20151231',
 '20160331',
 '20160630',
 '20160930',
 '20161231',
 '20170331',
 '20170630',
 '20170930',
 '20171231',
 '20180331',
 '20180630',
 '20180930',
 '20181231',
 '20190331',
 '20190630',
 '20190930',
 '20191231',
 '20200331',
 '20200630',
 '20200930',
 '20201231',
 '20210331',
 '20210630',
 '20210930',
 '20211231']